In [67]:
import Utils.PTModel.Training as Training
import Utils.PTModel.Inference as Inference
import Utils.Data.DataLoading as DataLoading
import Utils.PTModel.Models as Models

import Utils.EvaluationMetrics.alphaScore as AS
import Utils.EvaluationMetrics.exampleBased as EB
import Utils.EvaluationMetrics.labelBased as LB

import numpy as np
import pandas as pd

MODELARCH = Models.TileEmbeddingVAE

In [59]:
crossValidationData = DataLoading.LoadCrossValTrainTestData("../data/tomData/unshuffled3x3tiles.csv", shuffle=True)

# order of games in the original papers table
ordering = ['supermariobros', 'kidicarus', 'megaman', 'loderunner', 'thelegendofzelda']

# translation for original papers table format
translation = ['SMB', 'Kid Icarus', 'Megaman', 'Lode Runner', 'LOZ']


In [68]:
print(f"--- Cross Validation Testing ---")

crossValDataframe = pd.DataFrame()

crossValDataframe.insert(0, "AS B 0.25 G 0.75", 0)
crossValDataframe.insert(0, "AS B 1.0 G 1.0", 0)
crossValDataframe.insert(0, "AS B 0.75 G 0.25", 0)

crossValDataframe.insert(0, "LB Accuracy", 0)
crossValDataframe.insert(0, "LB Recall", 0)
crossValDataframe.insert(0, "LB Precision", 0)

crossValDataframe.insert(0, "EB Accuracy", 0)
crossValDataframe.insert(0, "EB Recall", 0)
crossValDataframe.insert(0, "EB Precision", 0)
crossValDataframe.insert(0, "EB EMR", 0)

crossValDataframe.insert(0, "Game Name", "")

for i, gamename in enumerate(ordering):

    k = gamename
    v = crossValidationData[gamename]

    print(f"\n{k}")
    print(f"trainDataShape: {v['trainData'].shape} | testDataShape: {v['testData'].shape}")

    model = Training.TrainModel(v, 50, 32, MODELARCH, earlyStoppingRange=2)

    yPredImage, yPredText = Inference.ModelInference(model, v['testData'])
    yTruthText = np.array(v['testData']["encodedAffordances"].to_list())

    as1 = AS.alpha_score(yTruthText, yPredText, beta=0.75, gamma=0.25)
    as2 = AS.alpha_score(yTruthText, yPredText, beta=1.0, gamma=1.0)
    as3 = AS.alpha_score(yTruthText, yPredText, beta=0.25, gamma=0.75)

    print(f"\nAlpha Score beta = 0.75 | gamma = 0.25: {AS.alpha_score(yTruthText, yPredText, beta=0.75, gamma=0.25)}")
    print(f"Alpha Score beta = 1.0 | gamma = 1.0: {AS.alpha_score(yTruthText, yPredText, beta=1.0, gamma=1.0)}")
    print(f"Alpha Score beta = 0.25 | gamma = 0.75: {AS.alpha_score(yTruthText, yPredText, beta=0.25, gamma=0.75)}")

    ebAcc = EB.example_based_accuracy(yTruthText, yPredText)
    ebPre = EB.example_based_precision(yTruthText, yPredText)
    ebRec = EB.example_based_recall(yTruthText, yPredText)
    ebEMR = EB.hamming_loss(yTruthText, yPredText)

    print(f"\nExample Based Accuracy: {EB.example_based_accuracy(yTruthText, yPredText)}")
    print(f"Example Based Precision: {EB.example_based_precision(yTruthText, yPredText)}")
    print(f"Example Based Recall: {EB.example_based_recall(yTruthText, yPredText)}")
    print(f"Example Based EMR: {EB.hamming_loss(yTruthText, yPredText)}")

    lbAcc = LB.accuracy_macro(yTruthText, yPredText)
    lbPre = LB.precision_macro(yTruthText, yPredText)
    lbRec = LB.recall_macro(yTruthText, yPredText)

    print("\nMacro")
    print(f"Label Based Accuracy: {LB.accuracy_macro(yTruthText, yPredText)}")
    print(f"Label Based Precision: {LB.precision_macro(yTruthText, yPredText)}")
    print(f"Label Based Recall: {LB.recall_macro(yTruthText, yPredText)}")

    crossValDataframe.loc[len(crossValDataframe)] = [translation[i], ebEMR, ebPre, ebRec, ebAcc, lbPre, lbRec, lbAcc, as1, as2, as3]

crossValDataframe.head()

--- Cross Validation Testing ---

supermariobros
trainDataShape: (19132, 7) | testDataShape: (860, 7)
Epoch 0: loss 6449.338123742553 | Val Text Loss: 427.1829833984375
Epoch 1: loss 3328.493397792446 | Val Text Loss: 478.1654357910156
Epoch 2: loss 2181.688084835193 | Val Text Loss: 997.5802001953125
Early Stopping...

Alpha Score beta = 0.75 | gamma = 0.25: 0.8316283924843424
Alpha Score beta = 1.0 | gamma = 1.0: 0.5544885177453027
Alpha Score beta = 0.25 | gamma = 0.75: 0.7228601252609603

Example Based Accuracy: 0.7098967247223061
Example Based Precision: 0.7235215902328491
Example Based Recall: 0.8174418807029724
Example Based EMR: 0.09543828264758497

Macro
Label Based Accuracy: 0.23680798709392548
Label Based Precision: 0.2899298369884491
Label Based Recall: 0.3448418080806732

kidicarus
trainDataShape: (19110, 7) | testDataShape: (882, 7)


/home/surfytom/Projects/Dissertation/Repos/TileEmbeddingDissertation/srcTom/Utils/EvaluationMetrics/labelBased.py:46: RuntimeWarning: invalid value encountered in divide
  sum_recall=np.nansum(p_n/p_d)


Epoch 0: loss 6308.980489571358 | Val Text Loss: 598.4029541015625
Epoch 1: loss 3292.014730906407 | Val Text Loss: 787.7969360351562
Epoch 2: loss 2135.4076123700092 | Val Text Loss: 1159.1510009765625
Early Stopping...

Alpha Score beta = 0.75 | gamma = 0.25: 0.7276203328834907
Alpha Score beta = 1.0 | gamma = 1.0: 0.41880341880341876
Alpha Score beta = 0.25 | gamma = 0.75: 0.6911830859199279

Example Based Accuracy: 0.5142668178382463
Example Based Precision: 0.5770975351333618
Example Based Recall: 0.5219199061393738
Example Based EMR: 0.11268096982382697

Macro
Label Based Accuracy: 0.15216980874538422
Label Based Precision: 0.30013635754585266
Label Based Recall: 0.15904465317726135

megaman
trainDataShape: (18523, 7) | testDataShape: (1469, 7)


/home/surfytom/Projects/Dissertation/Repos/TileEmbeddingDissertation/srcTom/Utils/EvaluationMetrics/exampleBased.py:32: RuntimeWarning: invalid value encountered in divide
  avg_precision=np.nansum(p_n/p_d)/float(n)
/home/surfytom/Projects/Dissertation/Repos/TileEmbeddingDissertation/srcTom/Utils/EvaluationMetrics/labelBased.py:16: RuntimeWarning: invalid value encountered in divide
  sum_acc=np.nansum(p_n/p_d)
/home/surfytom/Projects/Dissertation/Repos/TileEmbeddingDissertation/srcTom/Utils/EvaluationMetrics/labelBased.py:31: RuntimeWarning: invalid value encountered in divide
  sum_precision=np.nansum(p_n/p_d)
/home/surfytom/Projects/Dissertation/Repos/TileEmbeddingDissertation/srcTom/Utils/EvaluationMetrics/labelBased.py:46: RuntimeWarning: invalid value encountered in divide
  sum_recall=np.nansum(p_n/p_d)


Epoch 0: loss 6087.420302211518 | Val Text Loss: 855.7540893554688
Epoch 1: loss 3151.6834607165506 | Val Text Loss: 1839.3642578125
Epoch 2: loss 2074.1567456602843 | Val Text Loss: 661.7741088867188
Epoch 3: loss 1324.2087213651298 | Val Text Loss: 1548.001220703125
Epoch 4: loss 847.1750114585864 | Val Text Loss: 1287.7359619140625
Epoch 5: loss 542.0838686559279 | Val Text Loss: 1369.09326171875
Epoch 6: loss 347.9484616874207 | Val Text Loss: 1265.7132568359375
Epoch 7: loss 210.20188184352736 | Val Text Loss: 2062.104248046875
Epoch 8: loss 119.59343676657669 | Val Text Loss: 1606.135498046875
Epoch 9: loss 66.55789102544439 | Val Text Loss: 1613.89013671875
Epoch 10: loss 37.32957363622794 | Val Text Loss: 1695.99365234375
Early Stopping...

Alpha Score beta = 0.75 | gamma = 0.25: 0.845268817204301
Alpha Score beta = 1.0 | gamma = 1.0: 0.4593548387096774
Alpha Score beta = 0.25 | gamma = 0.75: 0.6140860215053763

Example Based Accuracy: 0.5942526500048624
Example Based Precision

...

Epoch 0: loss 9593.093470523232 | Val Text Loss: 1109.347900390625
Epoch 1: loss 6263.152687474301 | Val Text Loss: 1245.4736328125
Epoch 2: loss 5072.857502184416 | Val Text Loss: 1449.602294921875
Early Stopping...

Alpha Score beta = 0.75 | gamma = 0.25: 0.6387912061623846
Alpha Score beta = 1.0 | gamma = 1.0: 0.3109181378881394
Alpha Score beta = 0.25 | gamma = 0.75: 0.6721269317257548

Example Based Accuracy: 0.45800607220645495
Example Based Precision: 0.48088136315345764
Example Based Recall: 0.47851628065109253
Example Based EMR: 0.14624574158598316

Macro
Label Based Accuracy: 0.08590047061443329
Label Based Precision: 0.10945815593004227
Label Based Recall: 0.15275153517723083

thelegendofzelda
trainDataShape: (18362, 7) | testDataShape: (1630, 7)


/home/surfytom/Projects/Dissertation/Repos/TileEmbeddingDissertation/srcTom/Utils/EvaluationMetrics/labelBased.py:16: RuntimeWarning: invalid value encountered in divide
  sum_acc=np.nansum(p_n/p_d)
/home/surfytom/Projects/Dissertation/Repos/TileEmbeddingDissertation/srcTom/Utils/EvaluationMetrics/labelBased.py:31: RuntimeWarning: invalid value encountered in divide
  sum_precision=np.nansum(p_n/p_d)
/home/surfytom/Projects/Dissertation/Repos/TileEmbeddingDissertation/srcTom/Utils/EvaluationMetrics/labelBased.py:46: RuntimeWarning: invalid value encountered in divide
  sum_recall=np.nansum(p_n/p_d)


Epoch 0: loss 6288.52478878008 | Val Text Loss: 847.5872802734375
Epoch 1: loss 3365.143336704799 | Val Text Loss: 1508.4190673828125
Epoch 2: loss 2197.40353404188 | Val Text Loss: 1198.27490234375
Epoch 3: loss 1433.26491033574 | Val Text Loss: 1088.753173828125
Epoch 4: loss 947.2516279586101 | Val Text Loss: 1399.4757080078125
Epoch 5: loss 623.2161905640925 | Val Text Loss: 1599.67236328125
Early Stopping...

Alpha Score beta = 0.75 | gamma = 0.25: 0.6802340211373931
Alpha Score beta = 1.0 | gamma = 1.0: 0.12304982385505792
Alpha Score beta = 0.25 | gamma = 0.75: 0.44281580271766485

Example Based Accuracy: 0.12229162793579972
Example Based Precision: 0.14085768163204193
Example Based Recall: 0.30674847960472107
Example Based EMR: 0.32892873997168476

Macro
Label Based Accuracy: 0.07557216286659241
Label Based Precision: 0.1010923907160759
Label Based Recall: 0.40215209126472473


/home/surfytom/Projects/Dissertation/Repos/TileEmbeddingDissertation/srcTom/Utils/EvaluationMetrics/labelBased.py:46: RuntimeWarning: invalid value encountered in divide
  sum_recall=np.nansum(p_n/p_d)


,Game Name,EB EMR,EB Precision,EB Recall,EB Accuracy,LB Precision,LB Recall,LB Accuracy,AS B 0.75 G 0.25,AS B 1.0 G 1.0,AS B 0.25 G 0.75
0,SMB,0.10,0.72,0.82,0.71,0.29,0.34,0.24,0.83,0.55,0.72
1,Kid Icarus,0.11,0.58,0.52,0.51,0.30,0.16,0.15,0.73,0.42,0.69
2,Megaman,0.13,0.61,0.94,0.59,0.34,0.46,0.30,0.85,0.46,0.61
3,Lode Runner,0.15,0.48,0.48,0.46,0.11,0.15,0.09,0.64,0.31,0.67
4,LOZ,0.33,0.14,0.31,0.12,0.10,0.40,0.08,0.68,0.12,0.44


In [69]:
pd.set_option("display.precision", 2)
crossValDataframe

,Game Name,EB EMR,EB Precision,EB Recall,EB Accuracy,LB Precision,LB Recall,LB Accuracy,AS B 0.75 G 0.25,AS B 1.0 G 1.0,AS B 0.25 G 0.75
0,SMB,0.10,0.72,0.82,0.71,0.29,0.34,0.24,0.83,0.55,0.72
1,Kid Icarus,0.11,0.58,0.52,0.51,0.30,0.16,0.15,0.73,0.42,0.69
2,Megaman,0.13,0.61,0.94,0.59,0.34,0.46,0.30,0.85,0.46,0.61
3,Lode Runner,0.15,0.48,0.48,0.46,0.11,0.15,0.09,0.64,0.31,0.67
4,LOZ,0.33,0.14,0.31,0.12,0.10,0.40,0.08,0.68,0.12,0.44


In [70]:
crossValDataframe.to_csv("test.csv", index=False)